In [116]:
#Import statements
import json
import requests
import pandas as pd
import numpy as np
import datetime
from pandas.io.json import json_normalize
from pathlib import Path
import os
import csv
import time
import demoji

In [117]:
# The stock Symbols that we will iterate through
# Apple = AAPL
# Amazon = AMZN
# Google = GOOGL
# Microsoft = MSFT
# Dell = DELL
# IBM = IBM
# Intel = INTC
# HP = HPQ
# Facebook = FB
# Cisco Systems = CSCO
# Oracle = ORCL
# HP Enterprise = HPE
# Micron Tech = MU
# DXC Tech = DXC
# Thermo Fisher Scientific = TMO
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

In [118]:
#This gets the parent directory of the current directory
current = os.getcwd()
path = Path(current)
twits_folder = str(path.parent)+'\\data\\StockTwits'
twits_folder

'C:\\Users\\Russell\\OneDrive - Marist College\\School\\Spring 2020\\DATA 450 Data Capping\\Repository\\data\\StockTwits'

In [119]:
#Gathers the json file and all needed data values.
def collect_New_Twits(res):
    try:
        
        #
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment.basic','symbols'}])
        
        #Reorders the columns
        df = df[['id','body','created_at','entities.sentiment.basic','symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment.basic': 'tag'})
        
    except:
        df = (json_normalize(res['messages'])[{'id','body','created_at','entities.sentiment','symbols'}])
               
        #Reorders the columns and gets rid of old symbols column
        df = df[['id','body','created_at','entities.sentiment', 'symbols']]
        
        #Renames the columns
        df = df.rename(columns = {'created_at':'created', 'entities.sentiment': 'tag'})
        

    #The following loops reformat the symbols column
    #It creates a list of dictionaries, {symbol: "symbol of company mentioned", title: "name of company"}
    dataList=[]
    
    for index in df['symbols']:
        symbolsList = []

        for diction in index:

            symbolsList.append({'symbol' : diction.get('symbol'), 'title' : diction.get('title')})
            
        dataList.append(symbolsList)
        
    df.insert(5, "newSymbols" ,dataList)
    
    df = df[['id','body','created','tag', 'newSymbols']]
    
    df = remove_emojis(df)

    #Replaces the NAN with a string "none"
    df = df.replace(np.nan, 'none', regex=True)
    
    #Replaces the string as a datetime variable
    dateFormat = "%Y-%m-%dT%H:%M:%SZ"
    df['created'] = pd.to_datetime(df['created'], format=dateFormat)
    
    df = df.sort_values(by = 'id', ascending = False)
    
    #returns the dataframe in correct format
    return df

In [120]:
##This function takes a Stocktwits dataframe and removes the emojis from the twit.
#This function then returns a dataframe with an extra column for the twit without emojis
def remove_emojis(dataframe):
    cleanSentList=[]
    
    for row in dataframe.body:
        cleanSentList.append(demoji.replace(row))
        
    dataframe.insert(5, 'cleanSents', cleanSentList)
    
    dataframe = dataframe[['id', 'cleanSents', 'created','tag', 'newSymbols']]
    
    return dataframe

In [ ]:
while True:
    for symbol in stockSymbol:
    
        #selects the file to add to
        file = twits_folder+'\\{}_twits.csv'.format(symbol)
    
        historicalTwits = pd.read_csv(file)
        newestID = historicalTwits['id'].iloc[0]
    
        url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(symbol)
        response = requests.get(url, params = {'since' : newestID}).json()
        
        if response['response']['status'] == 429:
            print("requests exceeded")
            time.sleep(600)
            continue
        
        if json_normalize(response['messages']).empty:
            time.sleep(15)
            time.sleep(15)
            continue
    
        tempTwitsDf = collect_New_Twits(response)

        newHistoricalTwits = tempTwitsDf.append(historicalTwits)
        
        newHistoricalTwits = newHistoricalTwits.drop_duplicates(subset = 'id')
    
        newHistoricalTwits.to_csv(file, index=False)
        time.sleep(15)
        #newHistoricalTwits.to_csv(r'{}_twitsCopy.csv'.format(symbol), index=False)
        time.sleep(15)

c:\users\russell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
c:\users\russell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
c:\users\russell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


In [78]:
response

{'response': {'status': 200},
 'symbol': {'id': 3012,
  'symbol': 'ORCL',
  'title': 'Oracle Corp.',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 12798},
 'cursor': {'more': False, 'since': 195795040, 'max': 195683209},
 'messages': [{'id': 195683209,
   'body': 'Oracle $ORCL BidaskScore is Upgraded to Sold https://bidaskclub.com/news/company-news/company-news-company-news/2020/02/oracle-corporation-orcl-bidaskscore-is-upgraded-to-sold/',
   'created_at': '2020-02-20T06:32:06Z',
   'user': {'id': 1177001,
    'username': 'BidaskBot',
    'name': 'BidaskBot',
    'avatar_url': 'https://avatars.stocktwits.com/production/1177001/thumb-1504377126.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/1177001/thumb-1504377126.png',
    'join_date': '2017-09-02',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 324,
    'following': 84,
    'ideas': 22846,
    'watchlist_stocks_count': 20,
    'like_count': 998,
    '

In [29]:
file = twits_folder+'\\{}_twits.csv'.format("INTC")
intelDF = pd.read_csv('https://raw.githubusercontent.com/rsbmarist/MaristYoungMen2020/master/data/StockTwits/INTC_twits.csv')

In [32]:
intelDF.to_csv(file, index=False)

In [22]:
#Without Apple because I tested the code on Apple already
#stockSymbol = ["AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
#               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

In [49]:
#Want to add a column with a clean twit
for symbol in stockSymbol:
    
        #selects the file to add to
    file = twits_folder+'\\{}_twits.csv'.format(symbol)
    
    historicalTwits = pd.read_csv(file)

    historicalTwits = historicalTwits.sort_values(by = 'id', ascending = False)
    
    historicalTwits = historicalTwits.drop_duplicates(subset = 'id')
    
    historicalTwits.to_csv(file, index=False)

In [51]:
    file = twits_folder+'\\{}_twits.csv'.format('DXC')
    
    historicalTwits = pd.read_csv(file)

In [58]:
response

{'response': {'status': 200},
 'symbol': {'id': 1471,
  'symbol': 'CSCO',
  'title': 'Cisco Systems, Inc.',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 38563},
 'cursor': {'more': True, 'since': 195259058, 'max': 195194515},
 'messages': [{'id': 195194515,
   'body': 'Your daily News digest for Cisco Systems $CSCO https://wsfriend.com/daily/news/CSCO/2020-02-16/',
   'created_at': '2020-02-16T21:29:56Z',
   'user': {'id': 732840,
    'username': 'WallStreetFriend',
    'name': 'Wall Street Friend',
    'avatar_url': 'https://avatars.stocktwits.com/production/732840/thumb-1577167895.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/732840/thumb-1577167895.png',
    'join_date': '2016-04-17',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 87,
    'following': 39,
    'ideas': 12614,
    'watchlist_stocks_count': 31,
    'like_count': 23,
    'plus_tier': '',
    'premium_room': '',
    'trade_app': False},

In [19]:
noEmojisDF.cleanSents

0        $CSCO  I would rather buy Cisco vs the rest of...
1        $AAPL great traders on here higher on below av...
2        $QCOM $AAPL \nHave u watched Korean movies or ...
3        $AAPL waiting waiting; for 330! I got all day ...
4        $AAPL This Pump today has my 330 calls fully E...
                               ...                        
90414    $AAPL THe BEST stock on this planet! $$$$$$$$$...
90415        $SPY $AMD $AAPL $JPM whats a monthly candle? 
90416    $FB lowest value among fang stocks.  Sell $AAP...
90417                               $AAPL  not sustainable
90418    $AAPL 4 Green weekly candles. Hasn&#39;t had m...
Name: cleanSents, Length: 90419, dtype: object

In [21]:
historicalTwits

,id,cleanSents,created,tag,newSymbols
0,194642803,$CSCO I would rather buy Cisco vs the rest of...,2020-02-12 21:59:02,none,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}, {'..."
1,194642760,$AAPL great traders on here higher on below av...,2020-02-12 21:58:51,Bearish,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}]"
2,194642633,$QCOM $AAPL \nHave u watched Korean movies or ...,2020-02-12 21:58:19,none,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}, {'..."
3,194642618,$AAPL waiting waiting; for 330! I got all day ...,2020-02-12 21:58:15,none,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}]"
4,194642405,$AAPL This Pump today has my 330 calls fully E...,2020-02-12 21:57:24,none,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}]"
...,...,...,...,...,...
90414,182248130,$AAPL THe BEST stock on this planet! $$$$$$$$$...,2019-11-01 17:40:39,Bullish,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}]"
90415,182248110,$SPY $AMD $AAPL $JPM whats a monthly candle?,2019-11-01 17:40:26,none,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}, {'..."
90416,182247963,$FB lowest value among fang stocks. Sell $AAP...,2019-11-01 17:39:30,Bullish,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}, {'..."
90417,182247807,$AAPL not sustainable,2019-11-01 17:38:32,Bearish,"[{'symbol': 'AAPL', 'title': 'Apple Inc.'}]"


In [107]:
url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json".format('DELL')
response = requests.get(url, params = {'since' : 195873489}).json()

In [108]:
response

{'response': {'status': 200},
 'symbol': {'id': 1579,
  'symbol': 'DELL',
  'title': 'Dell Inc.',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 2971},
 'cursor': {'more': False, 'since': None, 'max': None},
 'messages': []}

In [115]:
json_normalize(response['messages']).empty

c:\users\russell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


True